Please install the pyrit version from the first cell. That only needs to be run once.

In [ ]:
# pip install pyrit==<insert commit hash>

Either change the folder_path below or put your images into that directory on your machine. "home" means "~"

In [ ]:
from pyrit.common import default_values
import pathlib
import json
import os

from pyrit.prompt_target.custom_image_to_text_target import CustomImageToTextTarget
from pyrit.prompt_normalizer.normalizer_request import NormalizerRequestPiece
from pyrit.prompt_normalizer.normalizer_request import NormalizerRequest
from pyrit.orchestrator import PromptSendingOrchestrator

default_values.load_default_env()

custom_image_to_text_target = CustomImageToTextTarget()

folder_path = pathlib.Path.home() / "git" / "custom_image_to_text"
data = [
    [
        {
            "prompt_text": str(folder_path / filename),
            "prompt_data_type": "image_path"
        }
    ]
    for filename in os.listdir(folder_path)
]
print(json.dumps(data, indent=4))

In [ ]:

normalizer_requests = []

for piece_data in data:
    request_pieces = []

    for item in piece_data:
        prompt_text = item.get("prompt_text", "")
        prompt_data_type = item.get("prompt_data_type", "")
        converters = []
        request_piece = NormalizerRequestPiece(
            prompt_text=prompt_text,
            prompt_data_type=prompt_data_type,
            prompt_converters=converters
        )
        request_pieces.append(request_piece)

    normalizer_request = NormalizerRequest(request_pieces)
    normalizer_requests.append(normalizer_request)

In [ ]:
import time

with PromptSendingOrchestrator(prompt_target=custom_image_to_text_target) as orchestrator:

    await orchestrator.send_normalizer_requests_async(prompt_request_list=normalizer_requests)

    memory_entries = orchestrator.get_memory()

    # write responses to CSV file
    orchestrator._memory.exporter.export_to_csv(
        data=[e for e in memory_entries if e.role == "assistant"],
        file_path=f"responses-{str(int(time.time()))}.csv")

    # print requests & responses to console
    for entry in memory_entries:
        print(entry)